# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [65]:
sc

Calculation started (calculation_id=0cc71cbf-af93-bb88-15a7-625bc10e4a5b) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [66]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=f6c71cbf-b6b9-1329-2ff6-6de640d33b51) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [67]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=bac71cbf-bc4a-858a-16f9-35c1ae5f083e) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [68]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'zhong61/zhong87'
print (user())

Calculation started (calculation_id=36c71cbf-bd76-6431-4073-282aea8058d9) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
zhong61/zhong87



## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [69]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    return trips.where(trips['trip_distance'] >= 2)
    pass

Calculation started (calculation_id=6ec71cbf-c2ab-9688-83e0-747d393cf057) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [70]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
     # Join the trips and lookup DataFrames to get the location information
    joined_df = trips.join(lookup, trips.DOLocationID == lookup.LocationID, 'inner')
    
    # Filter for locations in Manhattan
    manhattan_df = joined_df.where(col('Borough') == 'Manhattan')
    
    # Group by DOLocationID and sum the passenger_count
    pcount_df = manhattan_df.groupBy('DOLocationID').agg(sum('passenger_count').alias('pcount'))
    
    # Order by pcount in descending order and select the top 20 rows
    top_20 = pcount_df.orderBy(col('pcount').desc()).limit(20).select('DOLocationID', 'pcount')
    
    return top_20
    pass

Calculation started (calculation_id=32c71cbf-c433-bd9a-c3a4-12122b21f094) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [71]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    # Calculate the average total_amount
    avg_total_amount = trips.select(avg('total_amount').alias('avg_total_amount')).collect()[0]['avg_total_amount']

    # Count the total number of trips
    total_trips = trips.count()

    # Count the number of trips ending in the top 20 destinations
    top_20_dest_trips = trips.join(mtrips, trips.DOLocationID == mtrips.DOLocationID, 'inner').count()

    # Calculate the weight based on the ratio of trips ending in the top 20 destinations
    weight = top_20_dest_trips / total_trips

    # Calculate the weighted profit for each PULocationID
    wp = trips.groupBy('PULocationID') \
              .agg((avg('total_amount') * weight).alias('weighted_profit'))

    return wp
    pass

Calculation started (calculation_id=9ac71cbf-c57d-63cd-3dd3-ac60b638fb2e) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [72]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
        # Join the weighted_profit DataFrame with the lookup DataFrame
    final = wp.join(lookup, wp.PULocationID == lookup.LocationID, 'inner') \
              .select(lookup.Zone, lookup.Borough, wp.weighted_profit) \
              .orderBy(col('weighted_profit').desc()) \
              .limit(20)
    
    return final
    pass

Calculation started (calculation_id=7ec71cbf-ca95-2451-7604-efea2d66a8bc) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [73]:
def show_final_output(final):
    print("\nTop 20 locations with the highest weighted profit:")
    final.show(truncate=False)

Calculation started (calculation_id=92c71cbf-cfbb-37c0-1ba9-99b1c7ffbb66) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [74]:
# Load the trips and lookup DataFrames
trips, lookup = load_data(size='large')

# Generate the mtrips DataFrame
mtrips = manhattan_trips(trips, lookup)

# Call the weighted_profit function with the trips and mtrips DataFrames
wp = weighted_profit(trips, mtrips)

# Call the final_output function with the wp and lookup DataFrames
final = final_output(wp, lookup)

# Print the schema of the final DataFrame
print("Schema of the final DataFrame:")
final.printSchema()

# Call the standalone show_final_output function
show_final_output(final)

final.write.csv("s3://cse6242-zhong61/test1")


Calculation started (calculation_id=30c71cbf-d4ee-0bf4-c92a-9d616b2d6c61) in (session=e0c71c94-18bd-20cc-92e5-0b915818b9b8). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Trip Count:  187203269
Schema of the final DataFrame:
root
 |-- Zone: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- weighted_profit: double (nullable = true)


Top 20 locations with the highest weighted profit:
+---------------------------------+-------------+------------------+
|Zone                             |Borough      |weighted_profit   |
+---------------------------------+-------------+------------------+
|Newark Airport                   |EWR          |50.376634752188615|
|Briarwood/Jamaica Hills          |Queens       |49.53384999561871 |
|NA                               |Unknown      |43.53971052304923 |
|Arden Heights                    |Staten Island|42.073540833852114|
|Rossville/Woodrow                |Staten Island|41.88258790179781 |
|Oakwood                          |Staten Island|37.778665700893995|
|Bloomfield/Emerson Hill          |Staten Island|37.53039334181933 |
|Baisley Park                     |Queens       |32.9